In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf 
from scipy import stats
import warnings 

# ACCUSED DATA

In [2]:
# ACCUSED DATA 
accused_df = pd.read_csv('./project_data_unified/complaints/complaints-accused.csv')

# Remove rows for which final_outcome = NaN
accused_df = accused_df.dropna(subset=['final_outcome'])
# Remove rows for which final_outcome = "Unknown" (pretty much same thing as NaN)
accused_df = accused_df.loc[accused_df['final_outcome'] != 'Unknown']
# Rename complaint_category as complaint_description, per reference data
accused_df = accused_df.rename(columns={'complaint_category':'complaint_descr'})
# Drop reccommended finding and outcome bacause (a) >50% NaN and (b) mostly redundant
accused_df = accused_df.drop(columns={'recc_finding', 'recc_outcome'})


# Combine like values  
accused_df['final_outcome'] = accused_df['final_outcome'].astype(str)
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('NO ACTION TAKEN', 'No Action Taken'))
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('**PENALTY NOT SERVED', 'Penalty Not Served'))
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('REPRIMAND', 'Reprimand'))
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('SEPARATION', 'Separation'))
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('REINSTATED COURT ACT', 'Reinstated by Court Action'))
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('RESIGNED -NOT SERVED', 'Resigned'))
accused_df['final_outcome'] = accused_df['final_outcome'].apply(lambda x: x.replace('REINSTATED POLICE BD', 'Reinstated by Police Board'))


# treat '15 Day Suspension' same as '15 DAY SUSPENSION', for each # day 
import re 
accused_df['final_outcome'] = accused_df['final_outcome'].astype(str)

def standardize_case(s):  
    # Check if the string contains a number
    if any(char.isdigit() for char in s):
        # Extract the numeric part of the string using regular expressions
        num_str = re.findall(r'\d+', s)[0]
        # Use regular expressions to replace the text part with a lowercased version
        text = re.sub(r'[a-zA-Z]+', lambda m: m.group(0).lower(), s.replace(num_str, ''))
        # Combine the standardized text and numeric parts of the string
        return num_str + ' ' + text.strip()
    else:
        return s

# Update final_outcome 
accused_df['final_outcome'] = accused_df['final_outcome'].apply(standardize_case)

# Preview data
print(accused_df.shape)
print((accused_df.isnull().sum()/accused_df.shape[0]).sort_values(ascending=False))
accused_df.head()

/var/folders/q1/m9fsct652l59y6ctvtwrwg7c0000gn/T/ipykernel_27806/913147169.py:2: DtypeWarning: Columns (3,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  accused_df = pd.read_csv('./project_data_unified/complaints/complaints-accused.csv')


(236031, 8)
final_finding      0.000644
UID                0.000000
complaint_descr    0.000000
complaint_code     0.000000
cr_id              0.000000
cv                 0.000000
final_outcome      0.000000
link_UID           0.000000
dtype: float64


,UID,complaint_descr,complaint_code,cr_id,cv,final_finding,final_outcome,link_UID
0,130469.0,SUPERVISOR RESPONSIBILITY: FAIL TO OBTAIN COM...,12D,C170981,1,NS,No Action Taken,20040.0
1,103495.0,EXCESSIVE FORCE: ARRESTEE - DURING ARREST,05A,C170981,1,NS,No Action Taken,7727.0
2,111845.0,EXCESSIVE FORCE: ARRESTEE - DURING ARREST,05A,C170981,1,NS,No Action Taken,17149.0
3,126148.0,EXCESSIVE FORCE: ARRESTEE - DURING ARREST,05A,C170981,1,NS,No Action Taken,271.0
4,127906.0,EXCESSIVE FORCE: ARRESTEE - DURING ARREST,05A,C170981,1,NS,No Action Taken,11004.0


In [3]:
# Filter out all the final_outcomes that DON'T count as discpline
disc_outcomes = accused_df['final_outcome'].value_counts().index.difference(['No Action Taken', 'Reprimand', 'RESIGNED -NOT SERVED', 
                                                              'SUSTAINED-NO PENALTY', 'Violation Noted', 'Penalty Not Served', 
                                                                'Reinstated by Police Board', 'Resigned'])

# Create the binary, discplined or not, outcome variable 
accused_df['disciplined'] = accused_df['final_outcome'].apply(lambda x: 1 if x in disc_outcomes else 0)

In [4]:
# Create sustained or not sustained binary (final_find == NS)
accused_df['final_finding'].value_counts()
accused_df['sustained'] = accused_df['final_finding'].apply(lambda x: 1 if x == 'SU' else 0)

# Keep in mind, final_finding has other categories 

In [5]:
# Complaint categories 
complaint_ids = pd.read_excel('./project_data_unified/Complaint_Categories.xlsx')
complaint_ids = complaint_ids.rename(columns={111:'complaint_code', 'CATEGORY':'complaint_category', 
                                             'ON / OFF DUTY':'on_off_duty'})
# Collapse Excessive use of force into use of force
complaint_ids['complaint_category'] = complaint_ids['complaint_category'].apply(lambda x: x.replace('Excessive Force', 'Use of Force'))
# Combine racial profiling and first amendment into 'Other', since so few n
complaint_ids['complaint_category'] = complaint_ids['complaint_category'].apply(lambda x: x.replace('Racial Profiling', 'Other'))
complaint_ids['complaint_category'] = complaint_ids['complaint_category'].apply(lambda x: x.replace('First Amendment', 'Other'))

# Add ON/OFF duty and complaint category to accused_df from reference data
accused_df = pd.merge(accused_df, complaint_ids.loc[:,['complaint_code', 'on_off_duty', 'complaint_category']])

/Users/ryanpayne/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:296: UserWarning: Failed to load a conditional formatting rule. It will be discarded. Cause: expected <class 'openpyxl.worksheet.cell_range.MultiCellRange'>
  warn(msg)


# ROSTER

In [6]:
# Police roster information

roster_df = pd.read_csv('./project_data_unified/roster/roster_1936-2017_2017-04.csv')
star_nums = ['star1', 'star2', 'star3', 'star4', 'star5', 'star6', 'star7', 'star8', 'star9', 'star10']
to_drop = ['last_name_NS','first_name_NS', 'middle_initial', 'middle_initial2', 'suffix_name','roster_1936-2017_2017-04_ID',
          'row_id', 'merge', 'birth_year', 'resignation_date', 'link_UID']+star_nums
# Drop columns
roster_df = roster_df.drop(columns=to_drop)

# Combine all police officers into one category
# Identify rows containing 'PO' and replace with 'POLICE OFFICER'
mask = roster_df['current_rank'].str.contains('PO') & ~roster_df['current_rank'].str.contains('POLICE OFFICER') & ~roster_df['current_rank'].str.contains('PO AS DETECTIVE') 
roster_df.loc[mask, 'current_rank'] = 'POLICE OFFICER'

# Manually rename renaming "PO" entries
roster_df['current_rank'] = roster_df['current_rank'].apply(lambda x: x.replace('P O ASSGN SEC SPEC', 'POLICE OFFICER'))
roster_df['current_rank'] = roster_df['current_rank'].apply(lambda x: x.replace('P.O. ASSIGNED AS HELICOPTER PILOT', 'POLICE OFFICER'))
roster_df['current_rank'] = roster_df['current_rank'].apply(lambda x: x.replace('DEP CHIEF', 'CHIEF'))

# Collapse the rest into Other
to_replace = roster_df['current_rank'].value_counts().index.difference(['POLICE OFFICER', 'PO AS DETECTIVE', 'UNKNOWN', 'COMMANDER',
                                                      'DEP CHIEF', 'CHIEF'])

replace_dict = {level: 'Other' for level in to_replace}
roster_df['current_rank'] = roster_df['current_rank'].replace(replace_dict)


# Reduce number of unit descriptions 
counts = roster_df['unit_description'].value_counts()
roster_df.loc[roster_df['unit_description'].isin(counts[counts < 50].index), 'unit_description'] = 'Other'

# Rename columns 
roster_df = roster_df.rename(columns={'gender':'officer_gender', 'race':'officer_race', 'birth_year':'officer_birth_year', 
                                     'current_age':'officer_age'})

# Combine Native American and Pacific Islander, for balance reasons
roster_df['officer_race'] = roster_df['officer_race'].replace('NATIVE AMERICAN/ALASKAN NATIVE','ASIAN/PACIFIC ISLANDER')
roster_df['officer_race'] = roster_df['officer_race'].replace('ASIAN/PACIFIC ISLANDER', 'ASIA_PAC/NATIV_AM')

# Remove NaN and consolidate unknowns 
roster_df['unit_description'] = roster_df['unit_description'].fillna('Unknown')
roster_df['unit_description'] = roster_df['unit_description'].apply(lambda x: x.replace('UNKNOWN', 'Unknown'))

# Consolidate traffic units
mask = roster_df['unit_description'].str.contains('TRAFFIC')
roster_df.loc[mask, 'unit_description'] = 'TRAFFIC RELATED'

# Consolidating Airport units 
mask = roster_df['unit_description'].str.contains('AIRPORT')
roster_df.loc[mask, 'unit_description'] = 'AIRPORT RELATED'

# Consolidating Public housing units 
mask = roster_df['unit_description'].str.contains('PUBLIC HOUSING')
roster_df.loc[mask, 'unit_description'] = 'PUBLIC HOUSING'

# Consolidating YOUTH and juvenile units 
mask = (roster_df['unit_description'].str.contains('YOUTH') | roster_df['unit_description'].str.contains('JUVENILE'))
roster_df.loc[mask, 'unit_description'] = 'YOUTH_JUVENILE'

# Consolidate gang units 
mask = roster_df['unit_description'].str.contains('GANG') 
roster_df.loc[mask, 'unit_description'] = 'GANG ENFORCEMENT_INVESTIGATION'

# Consolidate patrol units 
mask = roster_df['unit_description'].str.contains('PATROL') 
roster_df.loc[mask, 'unit_description'] = 'PATROL'

# Consolidate forensic units 
mask = roster_df['unit_description'].str.contains('FORENSIC') 
roster_df.loc[mask, 'unit_description'] = 'FORENSIC'

# Consolidate property units 
mask = roster_df['unit_description'].str.contains('PROPERTY') |  roster_df['unit_description'].str.contains('ASSET')
roster_df.loc[mask, 'unit_description'] = 'PROPERTY RELATED'

# Consoldiate minority units again, keep district 013 separate
counts = roster_df['unit_description'].value_counts()
roster_df.loc[roster_df['unit_description'].isin(counts[counts < 50].index), 'unit_description'] = 'Other'

print(roster_df.shape)
print((roster_df.isnull().sum()/roster_df.shape[0]).sort_values(ascending=False))
roster_df.head()

(32446, 13)
current_unit        0.027492
UID                 0.020033
old_UID             0.020033
officer_race        0.002065
appointed_date      0.000185
officer_age         0.000062
first_name          0.000031
last_name           0.000031
officer_gender      0.000000
current_status      0.000000
rank_no             0.000000
current_rank        0.000000
unit_description    0.000000
dtype: float64


,officer_gender,officer_race,officer_age,current_status,appointed_date,rank_no,current_rank,current_unit,unit_description,first_name,last_name,UID,old_UID
0,MALE,WHITE,46.0,1,2005-09-26,9171,POLICE OFFICER,16.0,DISTRICT 016,JEFFERY,AARON,112683.0,1.0
1,FEMALE,HISPANIC,36.0,1,2005-09-26,9161,POLICE OFFICER,15.0,DISTRICT 015,KARINA,AARON,116324.0,2.0
2,MALE,WHITE,75.0,0,1970-06-15,9161,POLICE OFFICER,543.0,DETACHED SERVICES - MISCELLANEOUS DETAIL,DANIEL,ABATE,104770.0,3.0
3,MALE,WHITE,74.0,0,1969-01-06,9165,PO AS DETECTIVE,640.0,DETECTIVE SECTION - AREA 4,CARMEL,ABBATE,103132.0,4.0
4,MALE,WHITE,86.0,0,1954-10-16,9171,POLICE OFFICER,25.0,DISTRICT 025,CARMEN,ABBATE,103145.0,5.0


# SALARY

In [7]:
# Salary data
salary_df = pd.read_csv('./project_data_unified/salary/salary_2002-2017_2017-09.csv')
salary_df['rank'] = salary_df['rank'].astype(str)

# Delete redundant columns 
salary_df = salary_df.filter(regex='^(?!salary-)')
salary_df = salary_df.drop(columns=['salary_2002-2017_2017-09_ID.1', 'salary_2002-2017_2017-09_ID', 'org_hire_date', 'year', 
                                   'spp_date', 'row_id'])

# Rename
salary_df['rank'] = salary_df['rank'].replace('POLICE OFFICER (ASSIGNED AS DETECTIVE)','PO AS DETECTIVE')

# Collapsing POLICE OFFICERS
mask = salary_df['rank'].str.contains('POLICE OFFICER') 
salary_df.loc[mask, 'rank'] = 'POLICE OFFICER'

# Collapsing other POLICE...
mask = salary_df['rank'].str.contains('POLICE') & ~salary_df['rank'].str.contains('POLICE OFFICER (ASSIGNED AS DETECTIVE)')
salary_df.loc[mask, 'rank'] = 'POLICE OTHER'

# RENAME
salary_df = salary_df.rename(columns={'POLICE OFFICER (ASSIGNED AS DETECTIVE)':'PO AS DETECTIVE'})

to_replace = salary_df['rank'].value_counts().index.difference(['POLICE OFFICER', 'SERGEANT', 'COMMANDER', 'CHIEF', 'DEPUTY CHIEF',
                                                                'LIEUTENANT', 'EXPLOSIVES TECHNICIAN I', 'POLICE OTHER', 'PO AS DETECTIVE'])
replace_dict = {level: 'Other' for level in to_replace}
salary_df['rank'] = salary_df['rank'].replace(replace_dict)

# Preview data
print(salary_df.shape)
print((salary_df.isnull().sum()/salary_df.shape[0]).sort_values(ascending=False))
salary_df.head()

/var/folders/q1/m9fsct652l59y6ctvtwrwg7c0000gn/T/ipykernel_27806/2650080424.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = salary_df['rank'].str.contains('POLICE') & ~salary_df['rank'].str.contains('POLICE OFFICER (ASSIGNED AS DETECTIVE)')


(212508, 8)
age_at_hire        0.014922
start_date         0.014851
pay_grade          0.000000
rank               0.000000
salary             0.000000
employee_status    0.000000
UID                0.000000
link_UID           0.000000
dtype: float64


,pay_grade,rank,salary,employee_status,start_date,age_at_hire,UID,link_UID
0,D|2A,PO AS DETECTIVE,64356.0,CAREER SERVICE,1990-08-27,24.0,100013,4443.0
1,D|1,POLICE OTHER,57426.0,CAREER SERVICE,1996-12-02,29.0,100014,4879.0
2,D|1,POLICE OTHER,61512.0,CAREER SERVICE,1986-11-17,21.0,100015,5520.0
3,D|1,POLICE OTHER,54672.0,CAREER SERVICE,1998-06-29,31.0,100016,5913.0
4,D|1,POLICE OTHER,52092.0,CAREER SERVICE,2000-02-28,33.0,100019,6388.0


In [8]:
# Merge roster and salary data
roster_sal_df = pd.merge(salary_df, roster_df, on='UID')
# # Drop rank, using rank information from roster_df as current_rank
# roster_sal_df = roster_sal_df.drop(columns='rank')
print(roster_sal_df.shape)
print(roster_sal_df.isnull().sum()/roster_sal_df.shape[0])
roster_sal_df.head()

(202053, 20)
pay_grade           0.000000
rank                0.000000
salary              0.000000
employee_status     0.000000
start_date          0.015011
age_at_hire         0.015085
UID                 0.000000
link_UID            0.000000
officer_gender      0.000000
officer_race        0.000510
officer_age         0.000000
current_status      0.000000
appointed_date      0.000000
rank_no             0.000000
current_rank        0.000000
current_unit        0.000876
unit_description    0.000000
first_name          0.000000
last_name           0.000000
old_UID             0.000000
dtype: float64


,pay_grade,rank,salary,employee_status,start_date,age_at_hire,UID,link_UID,officer_gender,officer_race,officer_age,current_status,appointed_date,rank_no,current_rank,current_unit,unit_description,first_name,last_name,old_UID
0,D|2A,PO AS DETECTIVE,64356.0,CAREER SERVICE,1990-08-27,24.0,100013,4443.0,MALE,BLACK,51.0,1,1990-08-27,9171,POLICE OFFICER,2.0,DISTRICT 002,AARON,CHATMAN,4443.0
1,D|2A,PO AS DETECTIVE,65646.0,CAREER SERVICE,1990-08-27,24.0,100013,4443.0,MALE,BLACK,51.0,1,1990-08-27,9171,POLICE OFFICER,2.0,DISTRICT 002,AARON,CHATMAN,4443.0
2,D|2A,PO AS DETECTIVE,65646.0,CAREER SERVICE,1990-08-27,24.0,100013,4443.0,MALE,BLACK,51.0,1,1990-08-27,9171,POLICE OFFICER,2.0,DISTRICT 002,AARON,CHATMAN,4443.0
3,D|2A,PO AS DETECTIVE,74946.0,CAREER SERVICE,1990-08-27,24.0,100013,4443.0,MALE,BLACK,51.0,1,1990-08-27,9171,POLICE OFFICER,2.0,DISTRICT 002,AARON,CHATMAN,4443.0
4,D|2A,PO AS DETECTIVE,77580.0,CAREER SERVICE,1990-08-27,24.0,100013,4443.0,MALE,BLACK,51.0,1,1990-08-27,9171,POLICE OFFICER,2.0,DISTRICT 002,AARON,CHATMAN,4443.0


In [9]:
# Merged officer roster and salary info with accused_df 
merged_df = pd.merge(accused_df, roster_sal_df, on='UID', how='inner')
# Drop duplicates
merged_df = merged_df.drop_duplicates(subset=['UID','cr_id', 'complaint_code'])
#Drop redundant
merged_df = merged_df.drop(columns='link_UID_y')
# Rename
merged_df = merged_df.rename(columns={'link_UID_x':'link_UID'})

print(merged_df.shape)
print((merged_df.isnull().sum()/merged_df.shape[0]).sort_values(ascending=False).head(5))
merged_df.head()

(194142, 30)
age_at_hire      0.013294
start_date       0.013289
final_finding    0.000680
officer_race     0.000299
current_unit     0.000113
dtype: float64


,UID,complaint_descr,complaint_code,cr_id,cv,final_finding,final_outcome,link_UID,disciplined,sustained,...,officer_age,current_status,appointed_date,rank_no,current_rank,current_unit,unit_description,first_name,last_name,old_UID
0,110583.0,SUPERVISOR RESPONSIBILITY: FAIL TO OBTAIN COM...,12D,C171915,1,SU,2 day suspension,30048.0,1,1,...,69.0,0,1969-12-01,9171,POLICE OFFICER,8.0,DISTRICT 008,HENRY,WARE,30048.0
2,110583.0,EXCESSIVE FORCE: ARRESTEE - DURING ARREST,05A,C154367,1,UN,No Action Taken,30048.0,0,0,...,69.0,0,1969-12-01,9171,POLICE OFFICER,8.0,DISTRICT 008,HENRY,WARE,30048.0
4,110583.0,VERBAL ABUSE - USE OF PROFANITY,01A,C256894,1,UN,No Action Taken,30048.0,0,0,...,69.0,0,1969-12-01,9171,POLICE OFFICER,8.0,DISTRICT 008,HENRY,WARE,30048.0
6,110583.0,USE OF PROFANITY,01A,284083,2,NS,No Action Taken,30048.0,0,0,...,69.0,0,1969-12-01,9171,POLICE OFFICER,8.0,DISTRICT 008,HENRY,WARE,30048.0
8,110583.0,COMMISSION OF CRIME: MISCELLANEOUS,08N,C188852,1,SU,SUSTAINED-NO PENALTY,30048.0,0,1,...,69.0,0,1969-12-01,9171,POLICE OFFICER,8.0,DISTRICT 008,HENRY,WARE,30048.0


In [10]:
# DISTRICT INFO FOR DEMOGRAPHIC CONTEXT DATA
merged_df['unit_description'] = merged_df['unit_description'].fillna('None')
district_values = merged_df[merged_df['unit_description'].str.contains('DISTRICT')]['unit_description']

In [11]:
## OFFICER ONLY DATA !!!! 
df = merged_df.copy()
df.to_csv('officers_only.csv')

# Final clean

### Creating dummy variables, filtering out un-usable variables, etc.

In [12]:
# Ignore regular expression warning messages
warnings.filterwarnings('ignore')

# DUMMIES::

# unit_description dummy df
unit_df = (pd.get_dummies(df['unit_description'])).astype(float)
unit_df = unit_df.drop(columns='Unknown')
# complaint_category dummy df
complaint_df = (pd.get_dummies(df['complaint_category'])).astype(float)
complaint_df = complaint_df.drop(columns='Other')
# officer gender dummy 
gender_df = pd.get_dummies(df['officer_gender'])
# drop female (only need 1 dummy to encode binary)
gender_df = gender_df.drop(columns='FEMALE')
# officer Race dummy 
race_df = pd.get_dummies(df['officer_race'])
race_df.columns = race_df.columns.str.replace('/','_')
# on/off duty dummy 
duty_df = pd.get_dummies(df['on_off_duty'])
# Drop OFF dummy (only need 1 to encode binary)
duty_df = duty_df.drop(columns='OFF')
# current rank of officer dummies 
df['current_rank'] = df['current_rank'].astype(str)
current_rank_df = pd.get_dummies(df['current_rank'])
current_rank_df.columns = current_rank_df.columns.str.replace(' ','_')
# drop UNKNOWN column (avoid perfect multicoll)
current_rank_df = current_rank_df.drop(columns='UNKNOWN')
# rename other column so we don't have 2 of them
current_rank_df = current_rank_df.rename(columns={'Other':'Other_rank'})

# Combine complaint_category + unit_description data frames 
complaint_unit_df = pd.concat([complaint_df, unit_df], axis=1)
# Combine gender and race 
gender_race_df = pd.concat([race_df, gender_df], axis=1)
# gender + race + unit + complaint category 
gender_race_unit_complaint_df = pd.concat([unit_df, complaint_df, gender_df, race_df], axis=1)
# Create full data frame 
full_df = pd.concat([gender_race_unit_complaint_df, duty_df, current_rank_df], axis=1)
# Add our 2 continuous variables
full_df['salary'] = df['salary']
full_df['officer_age'] = df['officer_age'] 
# Add disciplined
full_df['y'] = df['disciplined']

# Clean up column names (make compatiable with smf.)
full_df.columns = full_df.columns.str.replace(' ', '_')
full_df.columns = full_df.columns.str.replace('/', '_')
full_df.columns = full_df.columns.str.replace(')', '')
full_df.columns = full_df.columns.str.replace('(', '_')
full_df.columns = full_df.columns.str.replace('-', '_')
full_df.columns = full_df.columns.str.replace('.', '')
full_df.columns = full_df.columns.str.replace('&', '_')

In [13]:
### EXPORT FULL DATA 
full_df.to_csv('cleaned_data.csv')

In [14]:
full_df.head()

,ADMIN_SCHOOL_SECURIT,AIRPORT_RELATED,BOMB_AND_ARSON_DIVISION,BUREAU_OF_INTERNAL_AFFAIRS,BUREAU_OF_ORGANIZED_CRIME,CENTRAL_DETENTION_UNIT,CENTRAL_INVESTIGATIONS_DIVISION,DET_DIV_ADMIN,DETACHED_SERVICES___GOVERMENT_SECURITY,DETACHED_SERVICES___MISCELLANEOUS_DETAIL,...,WHITE,ON,CHIEF,COMMANDER,Other_rank,PO_AS_DETECTIVE,POLICE_OFFICER,salary,officer_age,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,80808.0,69.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,80808.0,69.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,80808.0,69.0,0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,80808.0,69.0,0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,1,80808.0,69.0,0
